In [2]:
import os

In [ ]:
# # 1) Remove ALL pyarrow copies
# !pip uninstall -y pyarrow apache-arrow arrow-cpp
# # 2) Hard-delete any stragglers that can shadow your install
# !rm -rf /root/.local/lib/python3.11/site-packages/pyarrow* \
#        /usr/local/lib/python3.11/dist-packages/pyarrow* \
#        /usr/lib/python3/dist-packages/pyarrow* \
#        /usr/lib/python3.11/dist-packages/pyarrow*
# !pip install --no-cache-dir "pyarrow==16.1.0" "pandas>=2.2.2" "datasets==2.20.0" "protobuf==4.25.3"


In [4]:
import pyarrow as pa, pandas as pd, datasets
print("pyarrow:", pa.__version__, "cpp:", pa.cpp_version)
import pyarrow.parquet

pyarrow: 21.0.0 cpp: 21.0.0


In [3]:
!pip install --quiet --upgrade datasets langchain langchain-community langchain-chroma sentence-transformers umap-learn matplotlib scikit-learn
!pip install --upgrade PyMuPDF
!pip install huggingface_hub
!pip install pandas requests beautifulsoup4 PyMuPDF datasets \
    langchain langchain-community langchain-chroma sentence-transformers \
    scikit-learn matplotlib torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 98.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 107.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# import fitz  # PyMuPDF
# from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from sklearn.metrics.pairwise import cosine_similarity

## Load Data

In [6]:
df_disease = pd.read_csv("/kaggle/input/scraped-output-metadata-new-csv/scraped_output_metadata_new.csv")
df_family = pd.read_csv("/kaggle/input/scraped-output/scraped_output_famresource_metadata.csv")
df_housing = pd.read_csv("/kaggle/input/scraped-output/scraped_output_housing_metadata_new.csv")
df_meal = pd.read_csv("/kaggle/input/scraped-output/scraped_output_meal_metadata_new.csv")
df_alldisease = pd.read_csv("/kaggle/input/scraped-output-alldisease-other/scraped_output_alldisease_metadata_new.csv")
df_other = pd.read_csv("/kaggle/input/scraped-output-alldisease-other/scraped_output_otherrelevant_metadata_new.csv")

In [82]:
pd.set_option("display.max_colwidth", 1000)

In [97]:
df = pd.concat([df_disease, df_family, df_housing, df_meal, df_alldisease, df_other], axis = 0)

In [8]:
df_patient = pd.read_csv("/kaggle/input/st-jude-patient-data/st_jude_patient_metadata.csv")

In [9]:
df_patient

,User,Status,Age,Diagnosis,Location
0,James P. Sullivan,Patient,7,Neuroblastoma,Domino’s Village
1,Helen Parr,"Primary caregiver/mother of Violet Carr, a pat...",35,SCID,Newly arriving for inpatient treatment
2,Colette Tatou,Patient,18,2nd AML Relapse,"Affiliate clinic in Hunstville, AL"
3,Miguel Rivera,Patient,9,Osteosarcoma,St. Jude Main Campus
4,Evelyn Abbott,Mother of Marcus Abbott,41,NaN,Domino’s Village
5,Marcus Abbott,Patient,6,Medulloblastoma,St. Jude Inpatient Unit
6,Jack-Jack Parr,Patient,2,SCID,NaN
7,Sophie LeClair,Patient,15,Acute Lymphoblastic Leukemia (ALL),"Affiliate clinic in Peoria, IL"
8,Robert Parr,Father of Violet Carr,38,NaN,Temporary housing near campus
9,Violet Carr,Patient,13,SCID,St. Jude Inpatient Unit


In [8]:
pd.set_option("display.max_colwidth", 1000)
# df = pd.read_csv("/kaggle/input/scraped-output-metadata-new-csv/scraped_output_metadata_new.csv")


In [10]:
import os
os.getcwd()

'/kaggle/working'

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5779 entries, 0 to 3563
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5779 non-null   int64 
 1   text        5778 non-null   object
 2   metadata    5779 non-null   object
dtypes: int64(1), object(2)
memory usage: 180.6+ KB


In [12]:
df.head()

,Unnamed: 0,text,metadata
0,0,Your child’s treatment for cancer depends on t...,{'source_url': 'https://together.stjude.org/en...
1,1,TreatmentsTreatments for childhood illnesses i...,{'source_url': 'https://together.stjude.org/en...
2,2,A new radiation treatment may treat brain tumo...,{'source_url': 'https://together.stjude.org/en...
3,3,Managing symptoms and side effects is importan...,{'source_url': 'https://together.stjude.org/en...
4,4,Coping with Side EffectsSide effects are unint...,{'source_url': 'https://together.stjude.org/en...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5779 entries, 0 to 3563
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5779 non-null   int64 
 1   text        5778 non-null   object
 2   metadata    5779 non-null   object
dtypes: int64(1), object(2)
memory usage: 180.6+ KB


In [101]:
import ast

df['metadata'] = df['metadata'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [89]:
df_meal['metadata'] = df_meal['metadata'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
df_meal['section']= df_meal['metadata'].apply(lambda x: x.get('section'))
df_meal['title']= df_meal['metadata'].apply(lambda x: x.get('title'))

In [94]:
df_meal['text'] = df_meal['title'] + " " + df_meal['section'] + " " + df_meal['text']
df_meal.drop(columns = ['combined_text'], inplace = True)

In [96]:
df_meal.drop(columns = ['section', 'title'], inplace = True)

In [98]:
df = pd.concat([df_disease, df_family, df_housing, df_meal, df_alldisease, df_other], axis = 0)

In [102]:
df[df['metadata'].apply(lambda x: x.get('section')) == 'Lunch']

,Unnamed: 0,text,metadata
3,3,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
6,6,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
9,9,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
12,12,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
15,15,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"


In [103]:
df = df.iloc[:, 1:]

In [104]:
df = df.dropna()

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5778 entries, 0 to 3563
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      5778 non-null   object
 1   metadata  5778 non-null   object
dtypes: object(2)
memory usage: 135.4+ KB


In [186]:
df_meal[df_meal['metadata'].apply(lambda x: x.get('section')).isin(['Breakfast', 'Lunch', 'Dinner'])]

,Unnamed: 0,text,metadata
2,2,Domino's Village Menu Breakfast Domino's Village Menu Breakfast Omelets\nScrambled Eggs\nWaffles\nSausage Patties\nTurkey Sausage Links\nBacon\nOatmeal\nGrits\nHashbrowns,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Breakfast', 'super_section': None}"
3,3,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
4,4,Domino's Village Menu Dinner Domino's Village Menu Dinner Pot Roast\nHerb Crusted Cod\nSteamed Rice\nButtered Egg Noodles\nSauté Vegetable Medley\nSteamed Broccoli,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Dinner', 'super_section': None}"
5,5,Domino's Village Menu Breakfast Domino's Village Menu Breakfast Omelets\nScrambled Eggs\nWaffles\nSausage Patties\nTurkey Sausage Links\nBacon\nOatmeal\nGrits\nHashbrowns,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Breakfast', 'super_section': None}"
6,6,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
7,7,"Domino's Village Menu Dinner Domino's Village Menu Dinner Soup: White Bean & Kale, Broccoli Cheddar\nChicken Jerk with Mango Salsa\nBaked Salmon Veracruz\nCarrots with Raisins & Almonds\nSanta Maria Pinto Beans\nPlantains\nTurnip Greens","{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Dinner', 'super_section': None}"
8,8,Domino's Village Menu Breakfast Domino's Village Menu Breakfast Omelets\nScrambled Eggs\nWaffles\nSausage Patties\nTurkey Sausage Links\nBacon\nOatmeal\nGrits\nHashbrowns,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Breakfast', 'super_section': None}"
9,9,Domino's Village Menu Lunch Domino's Village Menu Lunch Fajitas- Chicken or Steak\nMexican Rice\nSauteed Vegetables,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Lunch', 'super_section': None}"
10,10,Domino's Village Menu Dinner Domino's Village Menu Dinner Butter Chicken\nVegetable Biryani\nBasmati Rice with Cumin\nVegetable Korma\nBread Garlic Naan,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Dinner', 'super_section': None}"
11,11,Domino's Village Menu Breakfast Domino's Village Menu Breakfast Omelets\nScrambled Eggs\nWaffles\nSausage Patties\nTurkey Sausage Links\nBacon\nOatmeal\nGrits\nHashbrowns,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Breakfast', 'super_section': None}"


In [187]:
df_meal_copy = df_meal.copy()
df_meal_copy['section'] = df_meal_copy['metadata'].apply(lambda x: x.get('section'))
df_meal_copy['section'].drop_duplicates()

0                                                 Domino's Village Menu
1                                                                Sunday
2                                                             Breakfast
3                                                                 Lunch
4                                                                Dinner
17                                                             Saturday
18         Isolation To-Go Menu - St. Jude Children’s Research Hospital
32                          Inpatient Room Service Daily Dinner Special
33                                                               Friday
36                                                               Monday
37                                                              Tuesday
38                                                            Wednesday
39                                                             Thursday
40            Room Service Menu - St. Jude Children’s Research H

In [190]:
df_meal_copy['section'].drop_duplicates().tolist()

["Domino's Village Menu",
 'Sunday',
 'Breakfast',
 'Lunch',
 'Dinner',
 'Saturday',
 'Isolation To-Go Menu - St. Jude Children’s Research Hospital',
 'Inpatient Room Service Daily Dinner Special',
 'Friday',
 'Monday',
 'Tuesday',
 'Wednesday',
 'Thursday',
 'Room Service Menu - St. Jude Children’s Research Hospital',
 'Kay Kafe Menu',
 'Meal Plans and Assistance - St. Jude Children’s Research Hospital',
 'Snack Bags - St. Jude Children’s Research Hospital',
 None,
 'Starbucks - St. Jude Children’s Research Hospital',
 'Contact Us - St. Jude Children’s Research Hospital',
 'More Resources',
 'Food on Campus',
 'Education and Support',
 'Resources for Families',
 'Phone Numbers | St. Jude Care & Treatment']

In [189]:
pd.set_option("display.max_rows", 100)
df_meal[~df_meal['metadata'].apply(lambda x: x.get('section')).isin(['Breakfast', 'Lunch', 'Dinner'])]

,Unnamed: 0,text,metadata
0,0,Domino's Village Menu Domino's Village Menu Domino's Village Menu Domino's Village Menu Mock-up - Room Service\nMeals served Monday through Friday,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Domino's Village Menu', 'super_section': None}"
1,1,Domino's Village Menu Sunday Domino's Village Menu Sunday Meals only served Monday through Friday.,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Sunday', 'super_section': None}"
17,17,Domino's Village Menu Saturday Domino's Village Menu Saturday Meals are only served Monday through Friday.,"{'source_url': 'https://na.eventscloud.com/ehome/index.php?eventid=384973&tabid=1245802&is_admin=1', 'retrieved_at': '2025-09-29T20:03:40.949603', 'title': 'Domino's Village Menu', 'section': 'Saturday', 'super_section': None}"
18,18,"Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu\nRoom Service HoursMonday-Friday: 7AM–9PMWeekends & holidays 7:30AM–9PM\nKay Kafe HoursMonday-Friday: 7AM–7PMWeekends & holidays: 7:30AM–7PM\nKay Kafe ExpressEvery day 7–9PM\nDial901-595-3000Please have your MRN and menu selection\r\nready.\nPick up your order in the Kay Kafe in the pickup area by the bakery.\nFor free delivery assistance from Concierge Services, Monday-Friday, 8:30 a.m.-4:30 p.m., dial901-595-4000\nPlease provide your MRN and clinic location.\nEnroll here","{'source_url': 'https://www.stjude.org/patient-experience/meals/isolation-to-go-ordering/isolation-to-go-menu.html#28de44b08a40e182917ae85db1b2f5ebf4bbe7f427c79bf5a4f28fdd44688614=3', 'retrieved_at': '2025-09-29T20:03:41.501669', 'title': 'Isolation To-Go Menu - St. Jude Children’s Research Hospital', 'section': 'Isolation To-Go Menu - St. Jude Children’s Research Hospital', 'super_section': None}"
19,19,"Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital FruitApple – red or greenTangerineBananaFresh fruit bowlApplesauceFruit cup – peach or Mandarin orange; no-added-sugar options availableYogurtTube – strawberryGreek – strawberry or blueberryLow fat – blueberry, strawberry, peach or vanillaYogurt parfait: vanilla yogurt layered with fresh berries and topped with granolaBreads & bakeryToast – white or wheatBagel with cream cheeseBlueberry muffinCinnamon rollCold cerealCheerios®Cinnamon Toast Crunch®Froot Loops®Frosted Flakes®Honey Nut Cheerios®Lucky Charms®Hot cerealOatmeal (add brown sugar)Grits (add cheese)Hot sidesBiscuit (add sausage gravy)Hash brownEggs – fried, scrambled or hard-boiledBaconTurkey baconPork sausage pattyTurkey sausage linkEntrée(Served with warm butter topping and breakfas...","{'source_url': 'https://www.stjude.org/patient-experience/meals/isolation-to-go-ordering/isolation-to-go-menu.html#28de44b08a40e182917ae85db1b2f5ebf4bbe7f427c79bf5a4f28fdd44688614=3', 'retrieved_at': '2025-09-29T20:03:41.501669', 'title': 'Isolation To-Go Menu - St. Jude Children’s Research Hospital', 'section': 'Isolation To-Go Menu - St. Jude Children’s Research Hospital', 'super_section': None}"
20,20,"Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital Isolation To-Go Menu - St. Jude Children’s Research Hospital . Sugar-free syrup available.)Pancake:two fluffy buttermilk pancakesWaffle:crisp and golden B

In [210]:
df_meal.iloc[:, 1:].to_csv("scraped_output_meal_metadata_new.csv")

## Document Indexing

### Chunking

In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Iterable, List

def chunk_text(
    text: str,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_text(text)

def build_chunks_dataframe(
    df: pd.DataFrame,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> pd.DataFrame:

    records = []

    for source_idx, row in df.iterrows():
        seg_text = row['text']
        seg_metadata = row['metadata']

        seg_chunks = chunk_text(seg_text, chunk_size, chunk_overlap)

        for chunk_idx, chunk in enumerate(seg_chunks):
            meta = seg_metadata.copy()
            meta['source_idx'] = source_idx
            meta['chunk_idx'] = chunk_idx

            records.append({'chunk_text': chunk,
                            'metadata': meta})

    return pd.DataFrame(records)

In [107]:
df_chunks = build_chunks_dataframe(df)

In [108]:
df_chunks

,chunk_text,metadata
0,"Your child’s treatment for cancer depends on the type of cancer they have and how advanced it is. Treatments may include chemotherapy, radiation therapy, surgery, targeted therapy, immunotherapy, and stem cell (bone marrow) transplant. Find trusted resources you can understand to help you as you make decisions.\nAllogeneic Stem Cell (Bone Marrow) Transplant\nAutologous Stem Cell (Bone Marrow) Transplant\nBrachytherapy\nCAR T-Cell Therapy\nChemotherapy\nImmunotherapy\nProton Beam Radiation\nRadiation Therapy\nStem Cell Transplant\nSurface Guided Radiation Therapy (SGRT)\nSurgery\nTargeted Therapy","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Learn about treatments for childhood and adolescent cancers', 'super_section': None, 'source_idx': 0, 'chunk_idx': 0}"
1,"TreatmentsTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nHow Treatments May Affect YouKnowing the facts about treatment and side effects can help you be informed about your care. Get the facts about how treatment works and how it may affect you now and later in life.\nKnowing the facts about treatment and side effects can help you be informed about your care. Get the facts about how treatment works and how it may affect you now and later in life.","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Featured resources', 'super_section': None, 'source_idx': 1, 'chunk_idx': 0}"
2,"Video LibraryListen to hopeful thoughts, medical explanations, and practical advice from families and care teams facing childhood and adolescent cancer, blood disorders, and other serious illnesses.\nListen to hopeful thoughts, medical explanations, and practical advice from families and care teams facing childhood and adolescent cancer, blood disorders, and other serious illnesses.","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Featured resources', 'super_section': None, 'source_idx': 1, 'chunk_idx': 1}"
3,A new radiation treatment may treat brain tumors with fewer side effects so patients can function better after treatment.,"{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Research Spotlight', 'super_section': None, 'source_idx': 2, 'chunk_idx': 0}"
4,"Managing symptoms and side effects is important to your child’s recovery and well-being. Learn more about cancer symptoms and treatment side effects.\nANC and Neutropenia\nAnemia\nAvascular Necrosis (Osteonecrosis)\nBlood Clots\nCentral Venous Line (Catheter) Complications\nChemo Brain\nConstipation\nCytokine Release Syndrome\nDehydration\nDelayed Wound Healing\nDelirium\nDiarrhea\nDysarthria\nExcessive Daytime Sleepiness, Hypersomnia, and Narcolepsy\nFatigue\nFever and Signs of Infection\nGraft Versus Host Disease\nHair Loss\nHearing Loss\nInfertility in Male Cancer and Stem Cell Transplant Patients\nInsomnia\nJaundice\nLoss of Appetite and Weight Loss\nMalignant Hyperthermia\nMouth and Throat Sores\nNausea and Vomiting During Cancer Treatment\nNeuropathic Pain\nPeripheral Neuropathy\nPosterior Fossa Syndrome","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-symptoms-side-effects.html', 'retrieved_at

In [109]:
all_chunks = df_chunks['chunk_text'].tolist()
all_metas = df_chunks['metadata'].tolist()

In [110]:
len(all_chunks)

6374

In [111]:
all_chunks[:5]

['Your child’s treatment for cancer depends on the type of cancer they have and how advanced it is. Treatments may include chemotherapy, radiation therapy, surgery, targeted therapy, immunotherapy, and stem cell (bone marrow) transplant. Find trusted resources you can understand to help you as you make decisions.\nAllogeneic Stem Cell (Bone Marrow) Transplant\nAutologous Stem Cell (Bone Marrow) Transplant\nBrachytherapy\nCAR T-Cell Therapy\nChemotherapy\nImmunotherapy\nProton Beam Radiation\nRadiation Therapy\nStem Cell Transplant\nSurface Guided Radiation Therapy (SGRT)\nSurgery\nTargeted Therapy',
 'TreatmentsTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nHow Treatments May Affect YouKnowing the facts about trea

In [112]:
all_metas[:5]

[{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html',
  'retrieved_at': '2025-09-29T18:09:04.907923',
  'title': 'Cancer Treatments   - Together by St. Jude™',
  'section': 'Learn about treatments for childhood and adolescent cancers',
  'super_section': None,
  'source_idx': 0,
  'chunk_idx': 0},
 {'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html',
  'retrieved_at': '2025-09-29T18:09:04.907923',
  'title': 'Cancer Treatments   - Together by St. Jude™',
  'section': 'Featured resources',
  'super_section': None,
  'source_idx': 1,
  'chunk_idx': 0},
 {'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html',
  'retrieved_at': '2025-09-29T18:09:04.907923',
  'title': 'Cancer Treatments   - Together by St. Jude™',
  'section': 'Featured resources',
  'super_section': None,
  'source_idx': 1,
  'chunk_idx': 1},
 {'source_url': 'https://together.stjude.org/en-us/conditions/

### Embedding

In [113]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Setting device to '{device}'")

Setting device to 'cuda'


In [114]:
from typing import List
# Use a pre-trained Sentence-BERT model to convert each chunk into a semantic vector
# embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedder = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

def embed_chunk_docs(chunk_docs: List[str],
                     model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                     normalize: bool = True):
    """
    Embed chunks of documents based on a given model name.
    """

    embedder = HuggingFaceEmbeddings(
    model_name= model_name,
    encode_kwargs={"normalize_embeddings": normalize})

    embeddings = embedder.embed_documents(chunk_docs)

    return embedder, embeddings

In [115]:
# embedder, embeddings = embed_chunk_docs(all_chunks, "intfloat/e5-large-v2", True)
# embedder, embeddings = embed_chunk_docs(all_chunks, "sentence-transformers/all-MiniLM-L6-v2", True)
embedder, embeddings = embed_chunk_docs(all_chunks, "sentence-transformers/paraphrase-MiniLM-L6-v2", True)

In [116]:
len(embeddings)

6374

In [117]:
len(embeddings[0])

384

### Store Embeddings in Chroma DB

In [30]:
# !rm -rf ./chroma_scifact

In [118]:
from langchain_chroma import Chroma
# from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
import time
from pathlib import Path
from typing import Optional, Union


def store_embed_in_db(texts: List[str],
    metas: List[dict],
    embedder: Optional[Embeddings] = None,
    persist_directory: Optional[Union[str, Path]] = None
) -> Chroma:
    """
    Store text chunks (and their embeddings) in a Chroma vector database.

    You must provide *either* `embeddings` (precomputed) *or* an `embedder`
    (model that implements .embed_documents).

    Args:
        texts: List of text chunks to index.
        embedder: Optional Embeddings object; used to compute embeddings on the fly.
        persist_directory: Directory where the Chroma DB will be saved.
            If None, a timestamped folder will be created.

    Returns:
        A Chroma vector store instance with your texts indexed.
    """

    if embedder is None:
        raise ValueError("You must supply an `embedder`")

    if persist_directory is None:
        ts = int(time.time())
        persist_directory = Path(f"./chroma_scifact_run_{ts}")
    persist_directory = Path(persist_directory)

    all_metas = df_chunks['metadata'].tolist()


    db = Chroma.from_texts(
        texts=texts,
        embedding=embedder,
        metadatas = metas,
        persist_directory=str(persist_directory)
    )

    print("Chunks successfully stored in Chroma vector DB.")

    return db

In [119]:
db = store_embed_in_db(all_chunks, all_metas, embedder = embedder)

Chunks successfully stored in Chroma vector DB.


In [120]:
col = db._collection  # the underlying chromadb Collection
data = col.get()

In [121]:
col

Collection(name=langchain)

In [122]:
len(data['documents'])

6374

In [123]:
meta_section_set = set()

for meta in all_metas:
    meta_section_set.add(meta.get('section', ''))

In [124]:
meta_section_set

{'Key points about astrocytoma',
 'Tips for poor appetite',
 'Key points about planning a child’s funeral or memorial service',
 'Treatment summary',
 'When to call the nurse',
 'Ways to help',
 'The Health Insurance Marketplace',
 'Coping with anxiety during illness',
 'How parenteral nutrition is given',
 'How to treat the flu',
 'What is HIV?',
 'How to care for someone with a respiratory illness',
 'School Program',
 'Risk factors for acute lymphoblastic leukemia',
 'Prognosis of infant ALL',
 'How to do a saline flush in the peripheral IV',
 'Proton Beam Radiation for Pediatric Cancer Patients - Together by St. Jude™',
 'What is a peripheral IV?',
 'Prognosis of acute lymphoblastic leukemia',
 'Co-pays',
 'Promote eye health',
 'Moving around with a nerve block',
 'Red blood cell (RBC) count',
 'Consent Forms - St. Jude Children’s Research Hospital',
 'Other factors that may increase the risk of vision problems',
 'How does ultrasound work?',
 'Risk factors for oligodendroglioma',

In [125]:
meta_super_section_set = set()

for meta in all_metas:
    meta_super_section_set.add(meta.get('super_section', ''))

In [126]:
meta_super_section_set

{None}

In [40]:
# all_metas.get('section', '')

## Query-Response Pipeline (RAG)

In [197]:
# Improved retrieval function with query expansion and better ranking
from langchain.vectorstores.base import VectorStoreRetriever
import re
from typing import List, Tuple, Dict, Any

def expand_query(query: str) -> List[str]:
    """Expand query with related terms for better retrieval"""
    expansions = {
        # Medical terms
        "symptoms": ["signs", "indications", "early signs", "clinical features"],
        "treatment": ["therapy", "management", "care", "intervention"],
        "side effects": ["adverse effects", "toxicities", "complications"],
        "diagnosis": ["testing", "screening", "evaluation"],
        "prognosis": ["outlook", "survival", "recovery chances"],
        "clinical trial": ["research study", "trial", "experimental therapy"],
        # Cancer / disease
        "cancer": ["tumor", "carcinoma", "malignancy", "oncology"],
        "leukemia": ["blood cancer", "ALL", "AML"],
        "lymphoma": ["hodgkin", "non-hodgkin"],
        "infection": ["illness", "disease", "condition", "virus", "bacteria"],
        # Resources
        "meal": ["food", "nutrition", "dining", "cafeteria", "diet"],
        "housing": ["lodging", "accommodation", "place to stay", "family housing"],
        "transportation": ["shuttle", "travel", "ride", "commute", "bus service"],
        "financial": ["cost", "expenses", "billing", "assistance", "funding"],
        "support": ["counseling", "resources", "services", "help", "aid"],
        "eat": ["restaurants", "menu"],
        # Hospital / logistics
        "admission": ["application", "check-in", "eligibility", "requirements"],
        "visitor": ["guest", "family member", "parent", "guardian"],
        "appointment": ["clinic visit", "consultation", "checkup", "follow-up"],
    }

    expanded_queries = [query]
    query_lower = query.lower()

    for key, synonyms in expansions.items():
        if key in query_lower:
            for synonym in synonyms:
                expanded_queries.append(query.replace(key, synonym, 1))

    return expanded_queries

def detect_metadata_filter(query: str) -> Optional[Dict[str, Any]]:
    """
    Look for trigger phrases in the query and return an appropriate
    metadata filter for your vector store.
    """
    q = query.lower()
    # 2) detect_metadata_filter() dictionary
    mapping = {
        # --- St. Jude navigation & new patients ---
        "new patient":      {"section": {"$eq": "Information for New Patients | St. Jude Care & Treatment"}},
        "first visit":      {"section": {"$eq": "Information for New Patients | St. Jude Care & Treatment"}},
    
        # --- Housing ---
        "housing":          {"section": {"$eq": "Housing and Patient Services - St. Jude Children’s Research Hospital"}},
        "target house":     {"section": {"$eq": "Housing and Patient Services"}},
        "tri delta":        {"section": {"$eq": "Housing and Patient Services"}},
        "domino's village": {"section": {"$eq": "Domino's Village Menu"}},
    
        # --- Food services / menus ---
        "kay kafe":         {"section": {"$eq": "Kay Kafe Menu"}},
        "kay cafe":         {"section": {"$eq": "Kay Kafe Menu"}},
        "room service":     {"section": {"$eq": "Room Service Menu - St. Jude Children’s Research Hospital"}},
        "isolation":        {"section": {"$eq": "Isolation To-Go Menu - St. Jude Children’s Research Hospital"}},
        "menu":             {"section": {"$eq": "Kay Kafe Menu"}},
        # "eat":              {"section": {"$eq": "Kay Kafe Menu | Breakfast | Lunch | Dinner"}},
    
        # --- Logistics ---
        "parking":          {"section": {"$eq": "Parking and Construction | St. Jude Care & Treatment"}},
        "directions":       {"section": {"$eq": "Parking and Construction | St. Jude Care & Treatment"}},
        "visitor":          {"section": {"$eq": "Visitors"}},
    
        # --- Patient portal & programs ---
        "mychart":          {"section": {"$eq": "St. Jude MyChart"}},
        "child life":       {"section": {"$eq": "Child Life | St. Jude Care & Treatment"}},
        "school program":   {"section": {"$eq": "The St. Jude School Program | St. Jude Care & Treatment"}},
    
        # --- Clinical trials / research ---
        "clinical trial":   {"section": {"$eq": "Clinical Trials | St. Jude Care & Treatment"}},
        "consent":          {"section": {"$eq": "Understanding and signing consent forms"}},
    
        # --- Policies / legal ---
        "hipaa":            {"title":   {"$eq": "Notice of Privacy Practices (HIPAA) - St. Jude Children’s Research Hospital"}},
        "privacy":          {"section": {"$eq": "Privacy & Legal"}},
        "nondiscrimination":{"section": {"$eq": "Discrimination is against the law"}},
    
        # --- Support services / community ---
        "concierge":        {"section": {"$eq": "Best Upon Request Concierge Service\u202f | St. Jude Care & Treatment"}},
        "st. jude voice":   {"section": {"$eq": "St. Jude Voice: Our Virtual Adviser community | St. Jude Care & Treatment"}},
    
        # --- Treatments ---
        "chemotherapy":     {"section": {"$eq": "Chemotherapy"}},
        "radiation":        {"section": {"$eq": "Radiation therapy"}},
        "surgery":          {"section": {"$eq": "Surgery"}},
        "targeted therapy": {"section": {"$eq": "Targeted therapy"}}
}

    # Regex-based mapping: if query contains both "eat" and "where"
    PAIR_RULES: List[Tuple[re.Pattern, List[str]]] = [
    # where + eat → any dining section
    (re.compile(r"\b(?:where|location|find|near|place|available|open|closest|nearby|there)\b.*\b(?:eat|food|meal|dining|cafe|kafe|menu|restaurant|cafeteria|kitchen|snack|coffee)\b"
    r"|"
    r"\b(?:eat|food|meal|dining|cafe|kafe|menu|restaurant|cafeteria|kitchen|snack|coffee)\b.*\b(?:where|location|find|near|place|available|open|closest|nearby|there)\b", re.I),
     ["Domino's Village Menu",
     'Sunday',
     'Breakfast',
     'Lunch',
     'Dinner',
     'Saturday',
     'Isolation To-Go Menu - St. Jude Children’s Research Hospital',
     'Inpatient Room Service Daily Dinner Special',
     'Friday',
     'Monday',
     'Tuesday',
     'Wednesday',
     'Thursday',
     'Room Service Menu - St. Jude Children’s Research Hospital',
     'Kay Kafe Menu',
     'Meal Plans and Assistance - St. Jude Children’s Research Hospital',
     'Snack Bags - St. Jude Children’s Research Hospital',
     'Starbucks - St. Jude Children’s Research Hospital',
     'Contact Us - St. Jude Children’s Research Hospital',
     'Food on Campus'])
]
    # regex_mapping = [
    #     (
    #         re.compile(r"\bwhere\b.*\beat\b|\beat\b.*\bwhere\b"),
    #         [
    #             {"section": {"$eq": "Breakfast"}},
    #             {"section": {"$eq": "Lunch"}},
    #             {"section": {"$eq": "Dinner"}},
    #         ]
    #     )
    # ]

    for trigger, filt in mapping.items():
        if trigger in q:
            return filt

    for pat, sections in PAIR_RULES:
        if pat.search(query):
            return {"section": {"$in": sections}}
    return None


def retrive_top_k_docs_improved(selected_query: str,
                               db_retriever: VectorStoreRetriever,
                               embedder: Embeddings,
                               k_docs: int = 10,  # Increased from 5
                               lambda_mult: float = 0.3,  # Lower for more similarity focus
                               query_prefix: str = "query: ",
                               doc_prefix: str = "passage: ",
                               use_query_expansion: bool = True):
    """
    Improved retrieval with query expansion and better re-ranking
    """

    if embedder is None:
        raise ValueError("You must supply an `embedder`.")

    # Step 1: Query expansion
    if use_query_expansion:
        expanded_queries = expand_query(selected_query)
    else:
        expanded_queries = [selected_query]

    # Step 2: Auto‐detect a metadata filter
    metadata_filter = detect_metadata_filter(selected_query)

    # Step 3: Retrieve documents for each expanded query with metadata filter
    all_retrieved_docs = []

    for exp_query in expanded_queries:
        formatted_query = f"{query_prefix}{exp_query}"

        # Define retriever method
        retriever = db_retriever.as_retriever(
            search_type="mmr",
            search_kwargs={"k": k_docs,
                           "lambda_mult": lambda_mult,
                          **({"filter": metadata_filter} if metadata_filter else {})}
        )

        # Get relevant documents
        results = retriever.get_relevant_documents(formatted_query)

        # Store documents with their expanded query context
        for doc in results:
            all_retrieved_docs.append((doc.page_content, exp_query))

    # Step 3: Remove duplicates while preserving order
    seen_docs = set()
    unique_docs = []
    for doc_content, exp_query in all_retrieved_docs:
        if doc_content not in seen_docs:
            seen_docs.add(doc_content)
            unique_docs.append((doc_content, exp_query))

    # Step 4: Re-rank all unique documents against original query
    original_formatted_query = f"{query_prefix}{selected_query}"
    query_embed = embedder.embed_query(original_formatted_query)

    # Format documents for embedding
    formatted_retrieved_docs = [f"{doc_prefix}{doc}" for doc, _ in unique_docs]

    # Embed documents
    doc_embeds = embedder.embed_documents(formatted_retrieved_docs)

    # Compute similarity scores
    similarity_scores = cosine_similarity([query_embed], doc_embeds)[0]

    # Step 5: Combine documents with scores and rank
    ranked_docs = []
    for i, (formatted_doc, score) in enumerate(zip(formatted_retrieved_docs, similarity_scores)):
        original_doc, exp_query = unique_docs[i]
        ranked_docs.append((formatted_doc, score, exp_query))

    # Sort by similarity score (descending)
    ranked_docs.sort(key=lambda x: x[1], reverse=True)

    # Step 6: Apply keyword boost for exact matches
    boosted_docs = []
    query_keywords = set(re.findall(r'\b\w+\b', selected_query.lower()))

    for doc, score, exp_query in ranked_docs:
        doc_keywords = set(re.findall(r'\b\w+\b', doc.lower()))
        keyword_overlap = len(query_keywords.intersection(doc_keywords))

        # Boost score based on keyword overlap
        boosted_score = score + (keyword_overlap * 0.05)  # Small boost for keyword matches
        boosted_docs.append((doc, boosted_score, exp_query))

    # Final sort by boosted scores
    boosted_docs.sort(key=lambda x: x[1], reverse=True)

    # Display results
    print("Query:", selected_query)
    print(f"\nExpanded to {len(expanded_queries)} queries")
    print(f"Retrieved {len(unique_docs)} unique documents")
    print(f"\nTop {k_docs} Retrieved Chunks with Scores:\n")

    final_results = []
    for i, (doc, score, exp_query) in enumerate(boosted_docs[:k_docs]):
        print(f"Chunk {i+1} (Score: {score:.4f}, From: '{exp_query}'):")
        print(f"{doc}")
        print(f"{'-'*80}")
        final_results.append((doc, score))

    return boosted_docs[:k_docs]

In [201]:
# Compare original vs improved retrieval with question selection
QUERIES = {"1": "What are brain and spinal cord tumors?",
           "2": "What are the most uncommon tumors?",
           "3": "What is adrenocortical tumor?",
           "4": "What is Family Commons at St. Jude?",
           "8": "I'm hungry, what is there to eat?",
           "9": "What are available restaurants?",
           "10": "Where is concierge?",
           "11": "Where can I get some tylenol?",
           "12": "How do I talk to my son about Violet being sick? He's having a hard time adjusting"}

print("Available queries:")
for key, value in QUERIES.items():
    print(f"{key}: {value}")

query_choice = input("Enter number of your query choice: ")
selected_query = QUERIES.get(query_choice)

if selected_query:
    print(f"\nSelected query: {selected_query}")

    print("\n" + "=" * 50)
    print("IMPROVED RETRIEVAL METHOD")
    print("=" * 50)

    result_improved = retrive_top_k_docs_improved(selected_query,
                       db_retriever = db,
                       embedder = embedder,
                       k_docs = 10,
                       lambda_mult = 0.3,
                       query_prefix = 'query: ',
                       doc_prefix = 'passage: ',
                       use_query_expansion = True)
else:
    print("Invalid choice. Please select 1, 2, or 3.")

Available queries:
1: What are brain and spinal cord tumors?
2: What are the most uncommon tumors?
3: What is adrenocortical tumor?
4: What is Family Commons at St. Jude?
8: I'm hungry, what is there to eat?
9: What are available restaurants?
10: Where is concierge?
11: Where can I get some tylenol?
12: How do I talk to my son about Violet being sick? He's having a hard time adjusting


Enter number of your query choice:  12



Selected query: How do I talk to my son about Violet being sick? He's having a hard time adjusting

IMPROVED RETRIEVAL METHOD
Query: How do I talk to my son about Violet being sick? He's having a hard time adjusting

Expanded to 1 queries
Retrieved 10 unique documents

Top 10 Retrieved Chunks with Scores:

Chunk 1 (Score: 0.7930, From: 'How do I talk to my son about Violet being sick? He's having a hard time adjusting'):
passage: It’s natural for children to feel uncomfortable when others see their brother or sister sick. People may ask questions about the illness that the siblings don’t know how to answer. Prepare children for questions and help them know what to say ahead of time. Help siblings know what to do in specific situations and encourage them to tell you when something makes them uncomfortable.
Consider whether you are comfortable with teachers or others at your children’s school talking with siblings about the illness. Let the school know how you prefer they communicate ab

In [42]:
df_patient

,User,Status,Age,Diagnosis,Location
0,James P. Sullivan,Patient,7,Neuroblastoma,Domino’s Village
1,Helen Parr,"Primary caregiver/mother of Violet Carr, a pat...",35,SCID,Newly arriving for inpatient treatment
2,Colette Tatou,Patient,18,2nd AML Relapse,"Affiliate clinic in Hunstville, AL"
3,Miguel Rivera,Patient,9,Osteosarcoma,St. Jude Main Campus
4,Evelyn Abbott,Mother of Marcus Abbott,41,NaN,Domino’s Village
5,Marcus Abbott,Patient,6,Medulloblastoma,St. Jude Inpatient Unit
6,Jack-Jack Parr,Patient,2,SCID,NaN
7,Sophie LeClair,Patient,15,Acute Lymphoblastic Leukemia (ALL),"Affiliate clinic in Peoria, IL"
8,Robert Parr,Father of Violet Carr,38,NaN,Temporary housing near campus
9,Violet Carr,Patient,13,SCID,St. Jude Inpatient Unit


In [50]:
patient_name = df_patient.iloc[:, 0]
patient_name.sample(n = 1, random_state = 123)

7    Sophie LeClair
Name: User, dtype: object

In [53]:
patient_name

0     James P. Sullivan
1            Helen Parr
2         Colette Tatou
3         Miguel Rivera
4         Evelyn Abbott
5         Marcus Abbott
6        Jack-Jack Parr
7        Sophie LeClair
8           Robert Parr
9           Violet Carr
10        Taro Watanabe
11          Maria Lopez
12          Lucia Lopez
13         Anjali Singh
Name: User, dtype: object

In [205]:
from langchain.vectorstores.base import VectorStoreRetriever

def create_rag_prompt(selected_query: str,
                       db_retriever: VectorStoreRetriever,
                       embedder: Embeddings,
                       k_docs: int = 5,
                       lambda_mult: float = 0.3,
                       query_prefix: str = "query: ",
                       doc_prefix: str = "passage: ",
                       use_query_expansion: bool = True):
    """
    Retrieve the top-k most relevant chunks for a given query using
    Maximal Marginal Relevance, then re-rank by cosine similarity.

    Args:
        query: Raw query string.
        db_retriever: The `db.as_retriever(...)` object configured with MMR.
        embedder: An Embeddings instance (implements embed_query & embed_documents).
        k: How many chunks to return.
        lambda_mult: MMR diversity parameter (0→pure similarity, 1→pure diversity).
        query_prefix: Text to prepend before embedding the query.
        doc_prefix: Text to prepend before embedding each chunk.

    Returns:
        A list of (chunk_text, score) sorted descending by score.
    """

    if embedder is None:
        raise ValueError("You must supply an `embedder`.")

    # Step 1: Query expansion
    if use_query_expansion:
        expanded_queries = expand_query(selected_query)
    else:
        expanded_queries = [selected_query]

    # Step 2: Auto‐detect a metadata filter
    metadata_filter = detect_metadata_filter(selected_query)

    # Step 3: Retrieve documents for each expanded query with metadata filter
    all_retrieved_docs = []

    for exp_query in expanded_queries:
        formatted_query = f"{query_prefix}{exp_query}"

        # Define retriever method
        retriever = db_retriever.as_retriever(
            search_type="mmr",
            search_kwargs={"k": k_docs,
                           "lambda_mult": lambda_mult,
                          **({"filter": metadata_filter} if metadata_filter else {})}
        )

        # Get relevant documents
        results = retriever.get_relevant_documents(formatted_query)

        # Store documents with their expanded query context
        for doc in results:
            all_retrieved_docs.append((doc.page_content, exp_query))

    # Step 4: Remove duplicates while preserving order
    seen_docs = set()
    unique_docs = []
    for doc_content, exp_query in all_retrieved_docs:
        if doc_content not in seen_docs:
            seen_docs.add(doc_content)
            unique_docs.append((doc_content, exp_query))

    # Step 5: Re-rank all unique documents against original query
    original_formatted_query = f"{query_prefix}{selected_query}"
    query_embed = embedder.embed_query(original_formatted_query)

    # Format documents for embedding
    formatted_retrieved_docs = [f"{doc_prefix}{doc}" for doc, _ in unique_docs]

    # Embed documents
    doc_embeds = embedder.embed_documents(formatted_retrieved_docs)

    # Compute similarity scores
    similarity_scores = cosine_similarity([query_embed], doc_embeds)[0]

    # Step 6: Combine documents with scores and rank
    ranked_docs = []
    for i, (formatted_doc, score) in enumerate(zip(formatted_retrieved_docs, similarity_scores)):
        original_doc, exp_query = unique_docs[i]
        ranked_docs.append((formatted_doc, score, exp_query))

    # Sort by similarity score (descending)
    ranked_docs.sort(key=lambda x: x[1], reverse=True)

    # Step 7: Apply keyword boost for exact matches
    boosted_docs = []
    query_keywords = set(re.findall(r'\b\w+\b', selected_query.lower()))

    for doc, score, exp_query in ranked_docs:
        doc_keywords = set(re.findall(r'\b\w+\b', doc.lower()))
        keyword_overlap = len(query_keywords.intersection(doc_keywords))

        # Boost score based on keyword overlap
        boosted_score = score + (keyword_overlap * 0.05)  # Small boost for keyword matches
        boosted_docs.append((doc, boosted_score, exp_query))

    # Final sort by boosted scores
    boosted_docs.sort(key=lambda x: x[1], reverse=True)

    # Combine the retrieved context into a single block of text
    context = "\n".join([doc for doc, score, exp_query in boosted_docs])

    # patient metadata
    n = 2
    patient_name = df_patient.iloc[n, 0]
    patient_status = df_patient.iloc[n, 1]
    patient_age = df_patient.iloc[n, 2]
    patient_diagnosis = df_patient.iloc[n, 3]
    patient_location = df_patient.iloc[n, 4]
    
    # Compose the RAG prompt by combining the context and the user's question
    rag_prompt = f"""
You are a helpful healthcare information assistant for families, patients, and caregivers.
You specialize in providing clear, accurate, and supportive information about childhood diseases,
cancer, user's conditions (if available) and St. Jude Children’s Research Hospital.

When answering:
- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).
- Present information in clear, compassionate language that non-experts can understand.
- Never provide personal medical advice, diagnoses, or treatment recommendations.
  Instead, encourage user to consult their doctors for medical concerns.

Answer structure:
If relevant information is found:
- Summarize the key facts clearly and accurately.
- Organize the response with short paragraphs, bullet points, or sections if helpful.
- Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).
- Please provide answers in English only
- Please limit your response into maximum 500 words
- Adjust your responses based on user profile, e.g. status (patient or parent), age, diagnosis, and additional information (if provided)
- For food-related questions, kindly check user status (patient or parent). If it is patient, kindly take into account additional information, such as patient allergy.
- For location-related questions, kindly take into account user location for your responses.
- For other non-health related questions, only provide answers from based on available context and user information. 
- If there is no available information, direct the user to St. Jude healthcare workers or relevant contact information.

Style:
- Use simple, reassuring language.
- Avoid technical jargon unless the retrieved text explicitly includes it.
- Keep the tone supportive and empathetic.
- Do not speculate or invent facts.

Remember: Your role is to **inform, not advise**. Always guide user toward
qualified healthcare providers for decisions about diagnosis or treatment.


    [CONTEXT]
    {context}

    [USER INFO]
    name: {patient_name}
    status: {patient_status}
    age: {patient_age}
    diagnosis: {patient_diagnosis}
    location: {patient_location}
    additional info: 
    """

    # Display the final composed prompt that will be passed to the LLM
    print("=== RAG Augmented Prompt ===")
    print(rag_prompt)

    return rag_prompt


In [206]:
# Define a new query
QUERIES = {"1": "What are the types of common tumors?",
           "2": "What are the most uncommon tumors?",
           "3": "What is adrenocortical tumor??",
           "4": "What is Family Commons at St. Jude?",
           "5": "What kinds of procedures might children with cancer undergo?",
           "6": "Will the MIBG scan hurt?", 
           "7": "What are the ingredients listed for the chicken tenders at the Kay Kafe?",
           "8": "I'm hungry, where to eat?", 
           "9": "What are available restaurants?",
           "10": "Where is concierge?",
           "11": "Where can I get some tylenol?",
           "12": "How do I talk to my son about Violet being sick? He's having a hard time adjusting",
           "13": "What happens if I just stop my treatment?"
           }

query_choice = input("Enter number of your query choice")
selected_query = QUERIES.get(query_choice)

rag_prompt = create_rag_prompt(selected_query,
                   db_retriever = db,
                   embedder = embedder,
                   k_docs = 10,
                   lambda_mult = 0.3,
                   query_prefix = 'query: ',
                   doc_prefix = 'passage: ',
                   use_query_expansion = True)

Enter number of your query choice 13


=== RAG Augmented Prompt ===

You are a helpful healthcare information assistant for families, patients, and caregivers.
You specialize in providing clear, accurate, and supportive information about childhood diseases,
cancer, user's conditions (if available) and St. Jude Children’s Research Hospital.

When answering:
- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).
- Present information in clear, compassionate language that non-experts can understand.
- Never provide personal medical advice, diagnoses, or treatment recommendations.
  Instead, encourage user to consult their doctors for medical concerns.

Answer structure:
If relevant information is found:
- Summarize the key facts clearly and accurately.
- Organize the response with short paragraphs, bullet points, or sections if helpful.
- Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).
- Please provide answers in Engli

## Gemma-3 1b

In [42]:
# !pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3 

from transformers import AutoTokenizer
from transformers.models.gemma3 import Gemma3ForCausalLM

In [43]:
GEMMA_PATH = "/kaggle/input/gemma-3/transformers/gemma-3-1b-it/1"
processor = AutoTokenizer.from_pretrained(GEMMA_PATH)
model = Gemma3ForCausalLM.from_pretrained(GEMMA_PATH)

In [44]:
# !pip install --upgrade --no-cache-dir "transformers>=4.44.2"

In [49]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": rag_prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is an adrenocortical tumor?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)


According to the text, an adrenocortical tumor (ACT) is a rare tumor of the adrenal gland that can be benign or malignant. It’s a tumor that can occur in the eye, head, or neck, and may spread to other parts of the body.otta Indoຫ
ຫ


In [51]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": rag_prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is Family Commons at St. Jude?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)

Family Commons at St. Jude is a treatment-free zone open to patients and families, offering a quiet space for relaxation and support. It’s a welcoming environment where you can find resources like Communication Needs, Child Life, and more, all designed to help you navigate your child’s care journey.

Here’s a breakdown of what Family Commons offers:

*   **Quiet Space:** A dedicated area for relaxation and respite between appointments.
*   **Resources:** Provides translation and interpretation, Child Life services, and social work support.
*   **Support Services:** Offers a range of services to assist families, including food services and a concierge service.

For more information, you can visit the St. Jude website at [https://www.stjude.org/](https://www.stjude.org/) or contact them at Patient & Family Resources.Experts can provide personalized guidance and support.
‌


In [53]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": rag_prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What kinds of procedures might children with cancer undergo?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)

Okay, here’s a response addressing your questions, based on the provided text:

As you’ve learned, childhood cancer is a rare disease, but it’s a significant concern. Here’s a breakdown of what’s important to know:

*   **Types of Childhood Cancers:** The most common types include leukemia, lymphoma, and brain tumors. There are many other types as well.
*   **Treatment Options:** Treatment options vary greatly depending on the type of cancer, its stage, and the child’s overall health. Common treatments include chemotherapy, surgery, radiation therapy, and targeted therapies.
*   **Ongoing Research:** Scientists are constantly working to develop new and more effective treatments for childhood cancers.
*   **Support and Resources:** It’s crucial to connect with your child’s medical team, and seek support from organizations like St. Jude Children’s Research Hospital.

Do you have any further questions about specific aspects of childhood cancer or treatment?reneianeyneeeeeeeeeeeeeeeeeeeeee

## Llama3.2 1b

In [207]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Use HF hub id or your local path
# e.g., LLAMA_PATH = "meta-llama/Llama-3.2-1B-Instruct"
# or    LLAMA_PATH = "/kaggle/input/llama-3-2-1b-instruct"
LLAMA_PATH = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(LLAMA_PATH, use_fast=True)
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model = AutoModelForCausalLM.from_pretrained(
    LLAMA_PATH,
    torch_dtype=dtype,
    device_map="auto" if torch.cuda.is_available() else None,
    trust_remote_code=True,  # safe for Meta's official weights
)

# Your RAG system prompt
# rag_prompt = "You are a helpful assistant that answers from retrieved context..."

messages = [
    {"role": "system", "content": [{"type": "text", "text": rag_prompt}]},
    {"role": "user",   "content": [{"type": "text", "text": selected_query}]},
]

# Build chat prompt and move tensors to model device
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Llama uses eos as pad; set it to avoid warnings
pad_id = tokenizer.eos_token_id

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,        # set True + temperature/top_p for sampling
        pad_token_id=pad_id,
    )

answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
print(answer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


I cannot provide medical advice. If you have concerns about your treatment, I suggest you speak with your healthcare provider or a qualified medical professional. Is there anything else I can help you with?


In [72]:
rag_prompt

"\nYou are a helpful healthcare information assistant for families, patients, and caregivers.\nYou specialize in providing clear, accurate, and supportive information about childhood diseases,\ncancer, user's conditions (if available) and St. Jude Children’s Research Hospital.\n\nWhen answering:\n- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).\n- Present information in clear, compassionate language that non-experts can understand.\n- Never provide personal medical advice, diagnoses, or treatment recommendations.\n  Instead, encourage user to consult their doctors for medical concerns.\n\nAnswer structure:\nIf relevant information is found:\n- Summarize the key facts clearly and accurately.\n- Organize the response with short paragraphs, bullet points, or sections if helpful.\n- Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).\n- Please provide answers in English only\n- P